In [ ]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns 

from sklearn.metrics import silhouette_score, silhouette_samples
import sklearn.metrics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

import itertools

import scipy

from datetime import datetime

#import sys
#!{sys.executable} -m pip install pandas-profiling
import pandas_profiling
#from yellowbrick.cluster import SilhouetteVisualizer, InterclusterDistance, KElbowVisualizer

#from kmodes.kmodes import KModes

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# This will ensure that matplotlib figures don't get cut off when saving with savefig()
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

# Reading Data and Viewing Data

In [ ]:
#Reading the csv files
points_data = pd.read_csv("Points.csv")
customer_data = pd.read_csv("CustomerDetail.csv")
fact_attribute_data = pd.read_csv("FactAttribute.csv")
customer_extension_data = pd.read_csv("CustomerExtension.csv")
quality_activity_data = pd.read_csv("QualityActivity.csv")
statistic_data=pd.read_csv("PointTypeStatistics.csv")
cineplex_data=pd.read_csv("OnlineStore.csv")
account_history=pd.read_csv("AccountHistory.csv")

# Feature Engineering

## 1 - Filtering and Aggregating data - Points Table

In [ ]:
#Filtering for Earned & Burned
points_earned = points_data[(points_data.points >= 0)]
points_burned = points_data[(points_data.points < 0)]

#Aggregating points and transactions by customer id
earned_agg = points_earned.groupby('Unique_member_identifier').agg(points_earned=pd.NamedAgg('points',sum))
burned_agg = points_burned.groupby('Unique_member_identifier').agg(points_burned=pd.NamedAgg('points',sum))
trans_agg = points_data.groupby('Unique_member_identifier').agg(trans_data=pd.NamedAgg('TransAmount',sum))


## 2 - Creating features to capture time

In [ ]:
from datetime import datetime

points_data['date'] = pd.to_datetime(points_data['pointdt'])
points_data.info()
points_data['hour']=points_data['date'].dt.hour

conditions = [points_data['hour'].between(5, 11), points_data['hour'].between(11, 17), points_data['hour'].between(17, 21),points_data['hour'].between(21, 24),points_data['hour'].between(0, 5)]
choices = ['morning','afternoon','evening','night','night']
points_data['time'] = np.select(conditions, choices)

#Filtering data by time
morn = points_data[(points_data.time =='morning')]
aft = points_data[(points_data.time =='afternoon')]
eve = points_data[(points_data.time =='evening')]
nig = points_data[(points_data.time =='night')]

#Counts for day/aft/eve/night transactions
morn_counts=morn.groupby('Unique_member_identifier').size().reset_index(name='mor_counts')
aft_counts=aft.groupby('Unique_member_identifier').size().reset_index(name='aft_counts')
eve_counts=eve.groupby('Unique_member_identifier').size().reset_index(name='eve_counts')
nig_counts=nig.groupby('Unique_member_identifier').size().reset_index(name='nig_counts')


## 3 -  Creating Features using Quality Table

In [ ]:
#Remove columns we don't need
quality_activity_data = quality_activity_data.drop(['ActivityMonth','isReachable','isSMS','hasActivity'],axis=1)

#Dummy variables
quality_activity_data = pd.get_dummies(quality_activity_data, columns=(['isQuality','isMarketable']), drop_first=False)

#Aggregate the number of trues and falses
quality_activity_data = quality_activity_data.groupby('Unique_member_identifier').agg('sum')

#lambda function to check which factor level has more occurences
quality_activity_data['Quality'] = quality_activity_data.apply(lambda X: 'False' if X.isQuality_False > X.isQuality_True else 'True',axis = 1)
quality_activity_data['Marketable'] = quality_activity_data.apply(lambda X: 'False' if X.isMarketable_False > X.isMarketable_True else 'True',axis = 1)

#Remove dummy columns before merging
quality_activity_data = quality_activity_data.drop(['isQuality_False','isMarketable_False','isQuality_True','isMarketable_True'],axis=1)
quality_activity_data

# 4 - Special Card Activity Feature

In [ ]:
#Dummy Variable for special card
special_card = [11,12,1252,1253,1254,1282,1283,1290,1322,1323]
special_card_activity=points_data.copy()
special_card_activity.loc[special_card_activity['pointtypeid'].isin(special_card), 'isspecial_card'] = 1
special_card_activity.loc[~special_card_activity['pointtypeid'].isin(special_card), 'isspecial_card'] = 0
special_card_final=special_card_activity[['Unique_member_identifier','isspecial_card']]

#Aggregating the count
special_card_final=special_card_final.groupby('Unique_member_identifier').agg('sum')

## 5 - Applogon - Mobile/web

In [ ]:
#App Login Codes
app_logon_id = [606,607]
app_logon_activity=account_history.copy()

#Creating dummy's for app login
app_logon_activity.loc[app_logon_activity['AccountHistoryTypeID'].isin(app_logon_id), 'app_web_logon'] = 1
app_logon_activity.loc[~app_logon_activity['AccountHistoryTypeID'].isin(app_logon_id), 'app_web_logon'] = 0
app_logon_final=app_logon_activity[['Unique_member_identifier','app_web_logon']]

#Aggregating the counts for app login
app_logon_final=app_logon_final.groupby('Unique_member_identifier').agg('sum')
app_logon_final

## 6 - Deleting customers with no activity and merging all datasets

In [ ]:
#Deleting customers with no activity
points_unique = pd.DataFrame(points_data['Unique_member_identifier'].unique())
points_unique.rename(columns = {0:'Unique_member_identifier'},inplace=True)
merged_data = pd.merge(customer_data,points_unique, on='Unique_member_identifier', how='right')

# Merging Features together

In [ ]:
#Combining features into customer detail dataframe
merged_data = pd.merge(merged_data,earned_agg, on='Unique_member_identifier', how='left')
merged_data = pd.merge(merged_data,burned_agg, on='Unique_member_identifier', how='left')
merged_data = pd.merge(merged_data,trans_agg, on='Unique_member_identifier', how='left')
merged_data = pd.merge(merged_data,morn_counts, on='Unique_member_identifier', how='left')
merged_data = pd.merge(merged_data,aft_counts, on='Unique_member_identifier', how='left')
merged_data = pd.merge(merged_data,eve_counts, on='Unique_member_identifier', how='left')
merged_data = pd.merge(merged_data,nig_counts, on='Unique_member_identifier', how='left')
merged_data = pd.merge(merged_data,quality_activity_data, on='Unique_member_identifier', how='left')
merged_data = pd.merge(merged_data,special_card_final, on='Unique_member_identifier', how='left')
merged_data = pd.merge(merged_data,app_logon_final, on='Unique_member_identifier', how='left')
merged_data=merged_data.fillna(0)


In [ ]:
merged_data.shape
merged_data.info()
merged_data.head(n=5)

# Combining Different Datasets

## 1 - Combining Fact Attribute, Customer Extension table with Customer Detail data

In [ ]:
merged_data = pd.merge(merged_data,fact_attribute_data, on='Unique_member_identifier', how='left')
merged_data = pd.merge(merged_data,customer_extension_data, on='Unique_member_identifier', how='left')
merged_data = merged_data.drop(['isActive','FSA','CreateDt','ReferredBy'], axis=1)
merged_data.head(2)

## 2 - Combining Type Statistic Table

In [ ]:
merged_data = pd.merge(merged_data,statistic_data, on='Unique_member_identifier', how='left')
merged_data = merged_data.drop(['BlackEarnCount','BlackBurnCount','Unique_member_identifier','BlackEarnLastDt','BlackEarnPointTotal','BlackBurnLastDt','BlackBurnPointTotal','LoadTime','OrderCount','OrderPointTotal','OrderLastDt','ConcessionLastDt','MusicStoreLastDt','BlackActivityDays','CnplxOnlineBonusLastDt','CnplxEarnTuesdayLastDt','LastDt','ChildTicketLastDt'], axis=1)
merged_data.head(2)

# FINAL DATASET 

In [ ]:
#FINAL DATSET FOR MODEL BUILDING
merged_data = merged_data.fillna(0)
df = merged_data.copy()
df.head(3)
df.info()
df.to_csv('check.csv')

# Hierarchial Model

## Step 1 - Scaling

In [ ]:
n_features = df.shape[1]

#Separating categorical and numerical variables
cat_col_names = list(df.select_dtypes(include=np.object).columns)
bool_col_names = list(df.select_dtypes(include=np.bool).columns)
num_col_names = list(df.select_dtypes(include=np.number).columns)

cat_col_names = cat_col_names + bool_col_names
X_num = df[num_col_names].to_numpy()
X_cat = df[cat_col_names].to_numpy()

#Scaling only the numerical variables
scaler = StandardScaler()
X_num = scaler.fit_transform(X_num)

X = np.concatenate((X_num, X_cat), axis=1)
X

## Step 2 - Distance Calculation

In [ ]:
#Defining a custom distance function for the heirarchial model to deal with categorical and numerical data
from scipy.spatial import distance

def custom_dist(x1, x2, cat_cols=None, num_cols=None):
   
    n = len(x1)

    dist_num = distance.euclidean(x1[num_cols], x2[num_cols])
    dist_cat = distance.hamming(x1[cat_cols], x2[cat_cols])
    
    dist = (dist_num*sum(num_cols) + dist_cat*sum(cat_cols)) / n
    
    return dist

In [ ]:
#Creating a custom distance matrix across all columns
from scipy.spatial.distance import pdist

cat_cols = [type(x)==str for x in X[1,:]]
num_cols = [not x for x in cat_cols]

Y = pdist(X, custom_dist, cat_cols=cat_cols, num_cols=num_cols)

## Step 3 - Hierarchial Clustering

In [ ]:
#Perform hierarchical clustering, using our custom-built distance matrix.
import scipy.cluster
aggl = scipy.cluster.hierarchy.linkage(Y, method='ward')

In [ ]:
#Plotting the Dendogram
plt.figure(figsize=(16, 10));
plt.grid(False)
dend = scipy.cluster.hierarchy.dendrogram(aggl);

In [ ]:
#Using k as 8 (as per business requirement)
K=8
labels = scipy.cluster.hierarchy.fcluster(aggl, K, criterion="maxclust")

## Step 4 - Viewing all Clusters

In [ ]:
#Viewing each cluster and its statistics

from scipy import stats
import seaborn as sns

pd.set_option("display.precision", 2)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print('All Data:')
print('Number of Instances: {}'.format(X.shape[0]))
df.describe(include=[np.number]).transpose()
df.describe(include=[np.object]).transpose()

for col in cat_col_names:
    df[col].value_counts()

for i, label in enumerate(set(labels)):
    n = df.iloc[labels==label].shape[0]
          
    print('\nCluster {}:'.format(label))
    print('Number of Instances: {}'.format(n))

    df.iloc[labels==label].describe(include=[np.number]).transpose()
    df.iloc[labels==label].describe(include=[np.object]).transpose()
    
    for col in cat_col_names:
        df.iloc[labels==label][col].value_counts()

## Saving numerical features into csv files

In [ ]:

n = df.iloc[labels==1].shape[0]
c1=df.iloc[labels==1].describe(include=[np.number]).transpose()
c1.to_csv('cluster1.csv')

n = df.iloc[labels==2].shape[0]
c1=df.iloc[labels==2].describe(include=[np.number]).transpose()
c1.to_csv('cluster2.csv')

n = df.iloc[labels==3].shape[0]
c1=df.iloc[labels==3].describe(include=[np.number]).transpose()
c1.to_csv('cluster3.csv')

n = df.iloc[labels==4].shape[0]
c1=df.iloc[labels==4].describe(include=[np.number]).transpose()
c1.to_csv('cluster4.csv')

n = df.iloc[labels==5].shape[0]
c1=df.iloc[labels==5].describe(include=[np.number]).transpose()
c1.to_csv('cluster5.csv')


n = df.iloc[labels==6].shape[0]
c1=df.iloc[labels==6].describe(include=[np.number]).transpose()
c1.to_csv('cluster6.csv')


n = df.iloc[labels==7].shape[0]
c1=df.iloc[labels==7].describe(include=[np.number]).transpose()
c1.to_csv('cluster7.csv')


n = df.iloc[labels==8].shape[0]
c1=df.iloc[labels==8].describe(include=[np.number]).transpose()
c1.to_csv('cluster8.csv')

In [ ]:
import dill
dill.dump_session("result_final.db")

In [ ]:
import dill
dill.load_session("result_final.db")